# Lista prática Extra

**Instruções gerais:** Sua submissão deve conter: 
1. Um "ipynb" com seu código e as soluções dos problemas
2. Uma versão pdf do ipynp

**Dica:** Considere usar o Google Colab

Essa lista terá o mesmo valor de uma questão na lista prática.

## Contexto
Neste exercício, você implementará um MLP simples para classificação binária e explorará aspectos fundamentais do PyTorch que frequentemente causam confusão: gradientes acumulados, inicialização de pesos, e o comportamento do modo de avaliação vs treinamento.

## Setup Inicial

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

# Configurar seed para reproducibilidade
torch.manual_seed(42)
np.random.seed(42)

# Gerar dataset toy
X, y = make_moons(n_samples=1000, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Converter para tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

## Parte 1: Implementação Básica do MLP

Implemente um MLP com a seguinte arquitetura:
- Entrada: 2 features
- Camada oculta 1: 16 neurônios, ativação ReLU
- Camada oculta 2: 8 neurônios, ativação ReLU  
- Saída: 1 neurônio (classificação binária com sigmoid)

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # TODO: Definir as camadas
        # self.fc1 = ...
        # self.fc2 = ...
        # self.fc3 = ...
        
    def forward(self, x):
        # TODO: Implementar o forward pass
        # Importante: NÃO aplique sigmoid na saída aqui - explicaremos por quê
        pass

## Parte 2: O Problema dos Gradientes Acumulados

Execute o código abaixo e explique por que os gradientes explodem:

In [ ]:
model = MLP()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Loop de treino INCORRETO - encontre o bug
for epoch in range(5):
    for i in range(len(X_train)):
        x = X_train[i:i+1]
        y = y_train[i:i+1]
        
        # NÃO adicione optimizer.zero_grad() aqui propositalmente
        
        output = torch.sigmoid(model(x))
        loss = F.binary_cross_entropy(output, y.unsqueeze(1))
        loss.backward()
        
        if i % 100 == 0:
            # Imprimir a norma do gradiente do primeiro peso
            grad_norm = model.fc1.weight.grad.norm().item()
            print(f"Epoch {epoch}, Step {i}, Grad norm: {grad_norm:.4f}")
    
    optimizer.step()


**Questão 2.1**: Por que os gradientes crescem indefinidamente? Corrija o código.

**Questão 2.2**: Onde exatamente você deve colocar `optimizer.zero_grad()` e `optimizer.step()`? Experimente colocar em lugares diferentes e observe o comportamento.

## Parte 3: Inicialização de Pesos

Compare diferentes estratégias de inicialização:

In [ ]:
def init_weights_zero(m):
    if isinstance(m, nn.Linear):
        m.weight.data.fill_(0.0)
        m.bias.data.fill_(0.0)

def init_weights_small(m):
    if isinstance(m, nn.Linear):
        m.weight.data.uniform_(-0.01, 0.01)
        
def init_weights_large(m):
    if isinstance(m, nn.Linear):
        m.weight.data.uniform_(-5, 5)

# Teste cada inicialização
init_functions = [init_weights_zero, init_weights_small, init_weights_large, None]
init_names = ['zeros', 'small', 'large', 'default (Xavier)']

for init_fn, name in zip(init_functions, init_names):
    model = MLP()
    if init_fn:
        model.apply(init_fn)
    
    # TODO: Treinar por 10 épocas e plotar a loss
    # Registre as activations da primeira camada oculta no início do treino
    
    with torch.no_grad():
        hidden1 = F.relu(model.fc1(X_train[:100]))
        print(f"\nInit {name}:")
        print(f"  Activations mean: {hidden1.mean():.4f}")
        print(f"  Activations std: {hidden1.std():.4f}")
        print(f"  Dead neurons: {(hidden1 == 0).all(dim=0).sum().item()}/16")

**Questão 3.1**: Por que a inicialização com zeros falha completamente?

**Questão 3.2**: Calcule manualmente a variância das activations esperada para cada inicialização. Como isso se relaciona com o problema do vanishing/exploding gradient?

## Parte 4: Modo de Treinamento vs Avaliação

Adicione Dropout e BatchNorm ao modelo:

In [ ]:
class MLPWithRegularization(nn.Module):
    def __init__(self):
        super(MLPWithRegularization, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.bn1 = nn.BatchNorm1d(16)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(16, 8)
        self.bn2 = nn.BatchNorm1d(8)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(8, 1)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

Execute o seguinte experimento:

In [ ]:
model = MLPWithRegularization()

# Passar o mesmo input 10 vezes SEM model.eval()
model.train()
outputs_train = []
x_single = X_test[0:1]

for _ in range(10):
    outputs_train.append(model(x_single).item())

print("Modo train - outputs:", outputs_train)
print("Desvio padrão:", np.std(outputs_train))

# Agora em modo eval
model.eval()
outputs_eval = []

for _ in range(10):
    outputs_eval.append(model(x_single).item())
    
print("\nModo eval - outputs:", outputs_eval)
print("Desvio padrão:", np.std(outputs_eval))

**Questão 4.1**: Por que os outputs variam em modo de treinamento mas são constantes em modo de avaliação?

**Questão 4.2**: Implemente uma função que estime a incerteza do modelo usando Monte Carlo Dropout (manter dropout ativo durante a inferência):

In [ ]:
def predict_with_uncertainty(model, x, n_samples=100):
    # TODO: Implementar predição com incerteza
    # Dica: Force model.train() mesmo durante inferência
    # Retorne média e desvio padrão das predições
    pass

## Parte 5: Investigando o Fluxo de Gradientes

Implemente hooks para visualizar gradientes durante backpropagation:

In [ ]:
gradients = {}

def save_gradient(name):
    def hook(grad):
        gradients[name] = grad
    return hook

model = MLP()

# Registrar hooks
x = torch.randn(1, 2, requires_grad=True)
x.register_hook(save_gradient('input'))

# Forward e backward
output = model(x)
output.backward()

# TODO: Adicionar hooks nas camadas intermediárias
# Plotar a magnitude dos gradientes em cada camada

**Questão 5.1**: Como a profundidade da rede afeta a magnitude dos gradientes? Experimente com redes de 2, 5 e 10 camadas.

**Questão 5.2**: Implemente gradient clipping e mostre como ele previne exploding gradients:

In [ ]:
# Durante o loop de treino
loss.backward()

# TODO: Implementar gradient clipping
# torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

optimizer.step()